In [2]:
import re
import pandas as pd
import json


In [3]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        for file in files:
            yield os.path.join(root, file)

# Example usage:
            
all_files = []
startpath = '../full_texts/'
for filepath in list_files(startpath):
    if ".json" in filepath:
        all_files.append(filepath)
        

In [4]:

def find_digital_twin_definitions(text):
    regex_patterns = [
        r'\b(?:survey)\b[^.]*?\b(?:digital\stwin)\b(?=\. )',
        r'\b(?:digital\stwin)\b[^.]*?\b(?:survey)\b(?=\. )',
        r'\b(?:digital\stwin)\b[^.]*?\b(?:review)\b(?=\. )',
        r'\b(?:review)\b[^.]*?\b(?:digital\stwin)\b(?=\. )',
    ]

    definitions = []
    for pattern in regex_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)  # Ignore case and match newline characters
        definitions.extend(matches)

    return definitions


In [5]:
all_defs = []
iter = 0
all_definitions = []
for index, file in enumerate(all_files):
    try:
        title = d["full-text-retrieval-response"]["coredata"]["dc:title"]
    except:
        title = "unkonwn"
    data ={
        "file": file,
        "title": title
    }
    file_defs = []
    try:
        with open( file, 'r') as f:
            d = f.read()
        definitions = find_digital_twin_definitions(d)
        if len(definitions) > 0 :
            for definition in definitions:
                if not definition in file_defs:
                    # print(iter, index,  definition)
                    file_defs.append(definition)
                    iter += 1
            data["defs"] = file_defs
            all_definitions.append(data)
            print(index,len(all_definitions),  end="\r")
    except Exception as e:
        pass

    with open("definitions_v2.json", 'w') as fw:
        fw.write(json.dumps(all_definitions))
    # if len(all_definitions) > 30:
    #     break

In [6]:
definitions_df = pd.DataFrame(all_definitions)
definitions_df

,file,title,defs
0,../full_texts/10.1016/j.ymssp.2022.109971/10.1...,unkonwn,"[digital twin: A systematic literature review,..."
1,../full_texts/10.1016/j.procir.2022.05.044/10....,unkonwn,[Digital twin paradigm: A systematic literatur...
2,../full_texts/10.1016/j.bmt.2023.02.001/10.101...,unkonwn,"[digital twin technologies in industrial, smar..."
3,../full_texts/10.1016/j.rser.2023.114138/10.10...,unkonwn,[digital twin application and the role of arti...
4,../full_texts/10.1016/j.procir.2021.03.078/10....,unkonwn,[Digital Twin: A systematic literature review]
...,...,...,...
330,../full_texts/10.1016/j.esg.2022.100147/10.101...,unkonwn,[Digital Twin: A systematic literature review]
331,../full_texts/10.1016/j.cie.2020.107094/10.101...,unkonwn,"[review (Kritzinger, Karner, Traar, Henjes, & ..."
332,../full_texts/10.1016/j.rcim.2019.101917/10.10...,unkonwn,[review This section reviews the related resea...
333,../full_texts/10.1016/j.cirp.2021.05.003/10.10...,unkonwn,[Digital Twin for maintenance: a literature re...


In [9]:
df_defs_exploded = definitions_df.explode("defs")

In [10]:
df_defs_exploded.drop_duplicates(subset="defs", inplace=True)
df_defs_exploded

,file,title,defs
0,../full_texts/10.1016/j.ymssp.2022.109971/10.1...,unkonwn,digital twin: A systematic literature review
1,../full_texts/10.1016/j.ymssp.2022.109971/10.1...,unkonwn,Digital twin paradigm: A systematic literature...
2,../full_texts/10.1016/j.ymssp.2022.109971/10.1...,unkonwn,Digital twin-based sustainable intelligent man...
4,../full_texts/10.1016/j.bmt.2023.02.001/10.101...,unkonwn,"digital twin technologies in industrial, smart..."
5,../full_texts/10.1016/j.bmt.2023.02.001/10.101...,unkonwn,Digital twin networks: A survey
...,...,...,...
492,../full_texts/10.1016/j.procir.2022.09.075/10....,unkonwn,"survey [2], current approaches hardly exploit ..."
495,../full_texts/10.1016/j.iintel.2023.100050/10....,unkonwn,Digital Twin technology for bridge maintenance...
499,../full_texts/10.1016/j.ifacol.2023.10.487/10....,unkonwn,Digital twin paradigm: A systematic litera- tu...
503,../full_texts/10.1016/j.cie.2020.107094/10.101...,unkonwn,"review (Kritzinger, Karner, Traar, Henjes, & S..."


In [11]:
df_defs_exploded = df_defs_exploded.reset_index(drop=True)
df_defs_exploded

,file,title,defs
0,../full_texts/10.1016/j.ymssp.2022.109971/10.1...,unkonwn,digital twin: A systematic literature review
1,../full_texts/10.1016/j.ymssp.2022.109971/10.1...,unkonwn,Digital twin paradigm: A systematic literature...
2,../full_texts/10.1016/j.ymssp.2022.109971/10.1...,unkonwn,Digital twin-based sustainable intelligent man...
3,../full_texts/10.1016/j.bmt.2023.02.001/10.101...,unkonwn,"digital twin technologies in industrial, smart..."
4,../full_texts/10.1016/j.bmt.2023.02.001/10.101...,unkonwn,Digital twin networks: A survey
...,...,...,...
165,../full_texts/10.1016/j.procir.2022.09.075/10....,unkonwn,"survey [2], current approaches hardly exploit ..."
166,../full_texts/10.1016/j.iintel.2023.100050/10....,unkonwn,Digital Twin technology for bridge maintenance...
167,../full_texts/10.1016/j.ifacol.2023.10.487/10....,unkonwn,Digital twin paradigm: A systematic litera- tu...
168,../full_texts/10.1016/j.cie.2020.107094/10.101...,unkonwn,"review (Kritzinger, Karner, Traar, Henjes, & S..."


In [13]:
text = ""
    i in df_defs_exploded.loc[:]["defs"]:
    text += i+"\n"
text

'digital twin: A systematic literature review\nDigital twin paradigm: A systematic literature review\nDigital twin-based sustainable intelligent manufacturing: A review\ndigital twin technologies in industrial, smart city and healthcare sectors: a survey\nDigital twin networks: A survey\nDigital Twin: A Survey\nDigital twin-based sustainable, intelligent manufacturing: A review\nDigital twin technology challenges and applications: A comprehensive review\ndigital twin application and the role of artificial intelligence in additive manufacturing: a systematic review\nDigital Twin: A systematic literature review\nDigital twin-based sustainable intelligent manufacturing: a review\nsurvey conducted by Erkoyuncu et al [5], it emerged that: i) much of the digital twin technology is already available in the market, but this has not been integrated yet to create a full digital twin; ii) the in-service phase can benefit more for the digital twin and; iii) there are many cultural challenges to th

In [15]:
lines = text.split('\n')

# Define a function to extract 100 characters before and after each result
def extract_context(result):
    match = re.search(r'(.{0,100}\b' + re.escape(result) + r'\b.{0,100})', text)
    return match.group(1) if match else None

# Iterate over each result and extract context
for line in lines:
    result = line.strip()
    context = extract_context(result)
    if context:
        print("Result:", result)
        print("Context:", context)
        print()

Result: digital twin: A systematic literature review
Context: digital twin: A systematic literature review

Result: Digital twin paradigm: A systematic literature review
Context: Digital twin paradigm: A systematic literature review

Result: Digital twin-based sustainable intelligent manufacturing: A review
Context: Digital twin-based sustainable intelligent manufacturing: A review

Result: digital twin technologies in industrial, smart city and healthcare sectors: a survey
Context: digital twin technologies in industrial, smart city and healthcare sectors: a survey

Result: Digital twin networks: A survey
Context: Digital twin networks: A survey

Result: Digital Twin: A Survey
Context: Digital Twin: A Survey

Result: Digital twin-based sustainable, intelligent manufacturing: A review
Context: Digital twin-based sustainable, intelligent manufacturing: A review

Result: Digital twin technology challenges and applications: A comprehensive review
Context: Digital twin technology challenge

In [20]:
def extract_context(x):
    with open(x["file"], 'r') as f:
        file = f.read()
    d = file.split(x["defs"])
    res = d[0][-100:]+ x["defs"] + d[1][:100]
    return res if res else None

    

df_defs_exploded["refs"] = df_defs_exploded.apply(lambda x : extract_context(x), axis = 1)

In [21]:
df_defs_exploded

,file,title,defs,refs
0,../full_texts/10.1016/j.ymssp.2022.109971/10.1...,unkonwn,digital twin: A systematic literature review,"access 8, 108952\u2013108971. [3] Jones D. Sn..."
1,../full_texts/10.1016/j.ymssp.2022.109971/10.1...,unkonwn,Digital twin paradigm: A systematic literature...,"al. Nature Computational Science 1, 313\u20133..."
2,../full_texts/10.1016/j.ymssp.2022.109971/10.1...,unkonwn,Digital twin-based sustainable intelligent man...,in paradigm: A systematic literature review. C...
3,../full_texts/10.1016/j.bmt.2023.02.001/10.101...,unkonwn,"digital twin technologies in industrial, smart...",ture and Future Directions. IEEE Network. [16]...
4,../full_texts/10.1016/j.bmt.2023.02.001/10.101...,unkonwn,Digital twin networks: A survey,orks: a survey IEEE Internet Things J. 8 18 20...
...,...,...,...,...
165,../full_texts/10.1016/j.procir.2022.09.075/10....,unkonwn,"survey [2], current approaches hardly exploit ...","igital twin, which provides a detailed reprodu..."
166,../full_texts/10.1016/j.iintel.2023.100050/10....,unkonwn,Digital Twin technology for bridge maintenance...,scanning: a review 2022 Adv. Civ. Eng. 2022 10...
167,../full_texts/10.1016/j.ifacol.2023.10.487/10....,unkonwn,Digital twin paradigm: A systematic litera- tu...,"/doi.org/10.1016/j.matpr.2022.02.566 Semeraro,..."
168,../full_texts/10.1016/j.cie.2020.107094/10.101...,unkonwn,"review (Kritzinger, Karner, Traar, Henjes, & S...",he digital twin has been adapted to deal with ...


In [28]:
all_refs = ""

for i in df_defs_exploded.loc[:]["refs"]:
    all_refs += i + "\n"
with open("refs.txt", 'w') as f:
    f.write(all_refs)